## Generating Indian Music using Pandas and Numpy

### Author : Vinay Kumar Thakur

In order to compose music, we should write notes in **compose.csv** file. 

The file **compose.csv** should have only two columns, the columns are 
 1. **half_beats**
 2. **notes**

* The value of half_beats can be: {1, 2, 3, 4, 5, 6, 7, 8}
* The value of notes can be one of the following
  (a) The notes in middle octaves can be written as following : 
         (i) suddha swar : {S, R, G, M, P, D, N}
         (ii) komal swar (flats) : {r, g, d, n}
         (iii) teevra swar (sharp) : {m}
         (iv) atirikta shruti : {r_a, R_a, g_a, G_a, M_a, m_a, d_a, D_a, n_a, N_a}
  (b) The swar in other octaves can be written as following :
         (i) mandra saptak (lower octaves): It is obtained by using prefix (l_) e.g l_S, l_R, l_g, l_m_a
         (ii) taar saptak (upper octaves): It is obtained by using prefix (u_)

Note :- ffmpeg(on linux) is needed to run this code

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd

In [3]:
TEMPO = 60 #beats_per_second
BASE_FREQ = 256
PI = np.pi

In [4]:
DELTA_CENT_PA = np.floor(1200 * np.log(3/2)/np.log(2) + 0.5)
DELTA_CENT_GA = np.floor(1200 * np.log(5/4)/np.log(2) + 0.5)

In [5]:
m_list = [idx - 3 for idx in range(10)]
n_list = [idx - 1 for idx in reversed(range(5))]

In [6]:
samvad = []
for n in n_list:
    temp = []
    for m in m_list:
        temp += [(m * DELTA_CENT_GA + n * DELTA_CENT_PA) % 1200]
    samvad += [temp]
    
samvad_df = pd.DataFrame(samvad)
samvad_df.columns = [f'C_{idx + 1}' for idx in range(len(samvad_df.columns))]
samvad_df.index = [f'R_{idx + 1}' for idx in range(len(samvad_df.index))]

In [7]:
notes_df = pd.DataFrame({'cents' : samvad_df.stack()})
notes_df.index.names = ['rows', 'columns']
notes_df.reset_index(inplace = True)
notes_df.sort_values(by = 'cents', inplace = True)
notes_df.eval('N = floor(cents/100)', inplace = True)

In [8]:
samvad_df

,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,C_10
R_1,948.0,134.0,520.0,906.0,92.0,478.0,864.0,50.0,436.0,822.0
R_2,246.0,632.0,1018.0,204.0,590.0,976.0,162.0,548.0,934.0,120.0
R_3,744.0,1130.0,316.0,702.0,1088.0,274.0,660.0,1046.0,232.0,618.0
R_4,42.0,428.0,814.0,0.0,386.0,772.0,1158.0,344.0,730.0,1116.0
R_5,540.0,926.0,112.0,498.0,884.0,70.0,456.0,842.0,28.0,414.0


In [9]:
octaves = []
for n in range(12):
    if n in [6, 11]:
        octaves += [notes_df.query('N == @n').iloc[[0]][['cents']]]
        continue
    
    octaves += [notes_df.query('N == @n').iloc[[0,-1]][['cents']]]
    
octaves_df = pd.concat(octaves, ignore_index = True)

In [10]:
octaves_df['notes'] = [
    'S',
    'r_a', 'r',
    'R_a', 'R',
    'g_a', 'g',
    'G', 'G_a',
    'M', 'M_a',
    'm', 'm_a',
    'P',
    'd_a', 'd',
    'D', 'D_a',
    'n_a', 'n',
    'N', 'N_a',
]

In [11]:
cents_df = octaves_df.set_index('cents')

In [12]:
octaves_df

,cents,notes
0,0.0,S
1,92.0,r_a
2,112.0,r
3,162.0,R_a
4,204.0,R
5,274.0,g_a
6,316.0,g
7,386.0,G
8,414.0,G_a
9,498.0,M


In [13]:
swar_samvad_df = (
    notes_df.sort_index()
            .merge(cents_df, how = 'left', left_on = 'cents', right_index = True)
            .pivot(index = 'rows', columns = 'columns', values = 'notes')
).loc[[f'R_{idx + 1}' for idx in range(5)], [f'C_{idx + 1}' for idx in range(10)]]

In [14]:
swar_samvad_df

columns,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,C_10
rows,,,,,,,,,,
R_1,NaN,NaN,M_a,D_a,r_a,NaN,NaN,NaN,NaN,NaN
R_2,NaN,NaN,n,R,m,n_a,R_a,NaN,NaN,NaN
R_3,NaN,NaN,g,P,N,g_a,NaN,NaN,NaN,m_a
R_4,NaN,NaN,d,S,G,d_a,NaN,NaN,NaN,N_a
R_5,NaN,NaN,r,M,D,NaN,NaN,NaN,NaN,G_a


In [15]:
three_octaves_df = pd.concat([
    pd.DataFrame({
        'notes': 'l_' + octaves_df['notes'],
        'cents': -1200 + octaves_df['cents']
    }),
    pd.DataFrame({
        'notes': octaves_df['notes'],
        'cents': octaves_df['cents']
    }),
    pd.DataFrame({
        'notes': 'u_' + octaves_df['notes'],
        'cents': 1200 + octaves_df['cents']
    }),
], ignore_index = True)

In [16]:
three_octaves_df.eval('freq = @BASE_FREQ * 2 ** (cents/1200)', inplace = True)

In [17]:
beats_df = pd.DataFrame({'n': np.arange(44100 * 10)})

In [18]:
beats_df.eval('t = n/44100', inplace = True)
beats_df.eval('beats = @TEMPO * t / 60', inplace = True)

In [19]:
 timbre_df = pd.concat([
     (
         beats_df.query(f'beats < {(n + 1) / 2}')
                  .eval(f'A = 3 * (beats / {(n + 1) / 2}) * exp( -9 * (beats / {(n + 1) / 2})**2)')
                  .eval(f'half_beats = {n + 1}')
     ) for n in range(8)
 ], ignore_index = True)

In [20]:
for freq, notes in three_octaves_df[['freq', 'notes']].values:
    timbre_df.eval(f'{notes} = A * sin(2 * @PI * {freq} * t)', inplace = True)

In [21]:
timbre_df.eval('rest = 0', inplace = True)

In [22]:
compose_df = pd.read_csv('compose.csv')

In [23]:
compose_df.head()

,half_beats,notes
0,1,G
1,1,G
2,1,G
3,1,N
4,1,D


In [24]:
track_df = pd.DataFrame({
    'y': pd.concat([
        timbre_df.query('half_beats == @b')[f'{n}'] 
        for _, (b, n) in compose_df[['half_beats', 'notes']].iterrows()
    ], ignore_index = True)
})

In [25]:
track_df = (track_df - track_df.min())/(track_df.max() - track_df.min())

In [26]:
track_df = ((2**16 - 1) * track_df + 0.5).apply(np.floor)

In [27]:
with open('music.raw', 'wb') as wf:
    wf.write(bytearray(np.uint16(track_df.values)))

In [28]:
!rm music.mp3 

In [29]:
!ffmpeg -f u16le -ar 44100 -ac 1 -i music.raw music.mp3

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib